# GMNS to AequilibraE example

## Inputs
1. Nodes as a .csv flat file in GMNS format
2. Links as a .csv flat file in GMNS format
3. Trips as a .csv flat file, with the following columns:  orig_node, dest_node, trips
4. Sqlite database used by AequilibraE

## Steps
1.  Read the GMNS nodes
-  Place in SQLite database, then translate to AequilibraE nodes
-  Generate the dictionary of zones for the omx trip table (uses node_type = centroid)

2. Read the GMNS links
-  Place in SQLite database, then translate to AequilibraE links

3. Read the trips
-  Translate into .omx file

A separate Jupyter notebook, Route, performs the following steps

4. Run AequilibraE shortest path and routing

5. Generate detail and summary outputs


In [1]:
#!/usr/bin/env python
# coding: utf-8
import os
import numpy as np
import pandas as pd
import sqlite3
#import shutil  # needed?
import openmatrix as omx
import math
#run_folder = 'C:/Users/Scott.Smith/GMNS/Lima'
run_folder = 'C:/Users/Scott/Documents/Work/AE/Lima'   #Change to match your local environment
#highest_centroid_node_number = 500   #we are now finding this from the nodes dataframe

## Read the nodes, and set up the dictionary of centroids
The dictionary of centroids is used later in setting up the omx trip table

In [2]:
#Read the nodes
node_csvfile = os.path.join(run_folder, 'GMNS_node.csv')
df_node = pd.read_csv(node_csvfile)   #data already has headers
print(df_node.head())   #debugging
df_size = df_node.shape[0]
print(df_size)

   node_id  name    x_coord    y_coord  z_coord node_type  ctrl_type  zone_id  \
0        1   NaN -84.106107  40.743322      NaN  centroid        NaN        1   
1        2   NaN -84.104302  40.743319      NaN  centroid        NaN        2   
2        3   NaN -84.102894  40.743284      NaN  centroid        NaN        3   
3        4   NaN -84.102975  40.740703      NaN  centroid        NaN        4   
4        5   NaN -84.104341  40.742056      NaN  centroid        NaN        5   

   parent_node_id  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
2232


In [3]:
# Set up the dictionary of centroids
# Assumption: the node_type = 'centroid' for centroid nodes
# The centroid nodes are the lowest numbered nodes, at the beginning of the list of nodes,
# but node numbers need not be consecutive
tazdictrow = {}
for index in df_node.index:
    if df_node['node_type'][index]=='centroid':
        #DEBUG print(index, df_node['node_id'][index], df_node['node_type'][index])
        tazdictrow[df_node['node_id'][index]]=index
#tazdictrow = {1:0,2:1,3:2,4:3,...,492:447,493:448}
taz_list = list(tazdictrow.keys())
matrix_size = len(tazdictrow)   #Matches the number of nodes flagged as centroids
print(matrix_size)   #DEBUG
highest_centroid_node_number = max(tazdictrow, key=tazdictrow.get)  #for future use
print(highest_centroid_node_number)  #DEBUG

449
493


## Read the links

In [4]:
# Read the links
link_csvfile = os.path.join(run_folder, 'GMNS_link.csv')
df_link = pd.read_csv(link_csvfile)   #data already has headers
#print(df_node.head())   #debugging
#df_size = df_link.shape[0]
print(df_link.shape[0])  #debug

6095


## Put nodes and links into SQLite.  Then translate to AequilibraE 0.6.5 format
1. Nodes are pushed into a table named GMNS_node
2. node table used by AequilibraE is truncated, then filled with values from GMNS_node
3. Centroid nodes are assumed to be the lowest numbered nodes, limited by the highest_centroid_node_number
    -  Number of centroid nodes must equal matrix_size, the size of the trip OMX Matrix
3. Links are pushed into a table named GMNS_link
4. link table used by AequilibraE is truncated, then filled with values from GMNS_link

### Some notes
1. All the nodes whole id is <= highest_centroid_node_number are set as centroids
2. GMNS capacity is in veh/hr/lane,  AequilibraE is in veh/hr;  hence, capacity * lanes in the insert statement
3. free_flow_time is assumed to be 60 * length / free_speed

In [5]:
#Open the Sqlite database, and insert the nodes and links
network_db = os.path.join(run_folder,'1_project','Lima.sqlite')
with sqlite3.connect(network_db) as db_con:
    #nodes
    df_node.to_sql('GMNS_node',db_con, if_exists='replace',index=False)
    db_cur = db_con.cursor()
    sql0 = "delete from nodes;"
    db_cur.execute(sql0)
    sql1 = ("insert into nodes(ogc_fid, node_id, x, y, is_centroid)" +
            " SELECT node_id, node_id, x_coord,y_coord,0 from " +
            " GMNS_node")
    db_cur.execute(sql1)
    sql2 = ("update nodes set is_centroid = 1 where ogc_fid <= " + str(highest_centroid_node_number))
    db_cur.execute(sql2)
    
with sqlite3.connect(network_db) as db_con:
    df_link.to_sql('GMNS_link',db_con, if_exists='replace',index=False)
    db_cur = db_con.cursor()
    sql0 = "delete from links;"
    db_cur.execute(sql0)
    sql1 = ("insert into links(ogc_fid, link_id, a_node, b_node, direction, distance, modes," +
            " link_type, capacity_ab, speed_ab, free_flow_time) " +
            " SELECT link_id, link_id, from_node_id, to_node_id, directed, length, allowed_uses," +
            " facility_type, capacity*lanes, free_speed, 60*length / free_speed" +
            " FROM GMNS_link where GMNS_link.capacity > 0")
    db_cur.execute(sql1)
    sql2 = ("update links set capacity_ba = 0, speed_ba = 0, b=0.15, power=4")
    db_cur.execute(sql2)

Next step is to update the links with the parameters for the volume-delay function.  This step is AequilibraE-specific and makes use of the link_types Sqlite table.  This table is taken from v 0.7.1 of AequilibraE, to ease future compatibility.  The link_types table expects at least one row with link_type = "default" to use for default values.  The user may add other rows with the real link_types.  

Its CREATE statement is as follows

```
CREATE TABLE 'link_types' (link_type     VARCHAR UNIQUE NOT NULL PRIMARY KEY,
                           link_type_id  VARCHAR UNIQUE NOT NULL,
                           description   VARCHAR,
                           lanes         NUMERIC,
                           lane_capacity NUMERIC,
                           alpha         NUMERIC,
                           beta          NUMERIC,
                           gamma         NUMERIC,
                           delta         NUMERIC,
                           epsilon       NUMERIC,
                           zeta          NUMERIC,
                           iota          NUMERIC,
                           sigma         NUMERIC,
                           phi           NUMERIC,
                           tau           NUMERIC)
```

| link_type | link_type_id | description | lanes | lane_capacity | alpha | beta | other fields not used |
| ----- | ----- | ----- | ----- | ----- |----- |----- |----- |
| default  | 99 | Default general link type | 2 | 900 | 0.15 | 4 | |

In [6]:
with sqlite3.connect(network_db) as db_con:
    db_cur = db_con.cursor()
    sql1 = "update links set b = (select alpha from link_types where link_type = links.link_type)"
    db_cur.execute(sql1)
    sql2 = ("update links set b = (select alpha from link_types where link_type = 'default') where b is NULL")
    db_cur.execute(sql2)
    sql3 = "update links set power = (select beta from link_types where link_type = links.link_type)"
    db_cur.execute(sql3)
    sql4 = ("update links set power = (select beta from link_types where link_type = 'default') where power is NULL")
    db_cur.execute(sql4)

## Read the trips, and translate to omx file

In [7]:
#Read a flat file trip table into pandas dataframe
trip_csvfile = os.path.join(run_folder, 'demand.csv')
df_trip = pd.read_csv(trip_csvfile)   #data already has headers
print(df_trip.head())   #debugging
df_size = df_trip.shape[0]
print(df_size)
#print(df.iloc[50]['o_zone_id'])
#stuff for debugging
print(df_trip['total'].sum())  #for debugging: total number of trips
#for k in range(df_size):   #at most matrix_size*matrix_size
#    i = tazdictrow[df_trip.iloc[k]['orig_taz']]
#    j = tazdictrow[df_trip.iloc[k]['dest_taz']]
#    if k == 4:  print(k,"  i=",i,"  j=",j)  #debugging

   orig_taz  dest_taz  total
0         1        57      1
1         1       138      1
2         2         2      1
3         2       287      1
4         3        67      1
13000
32041


In [8]:
#Write the dataframe to an omx file
# This makes use of tazdictrow and matrix_size, that was established earlier. 
# The rows are also written to a file that is used only for debugging
outfile = os.path.join(run_folder, '0_tntp_data' ,'demand.omx') 
outdebugfile = open(os.path.join(run_folder,'debug_demand.txt'),"w")
output_demand = np.zeros((matrix_size,matrix_size))
f_output = omx.open_file(outfile,'w')

f_output.create_mapping('taz',taz_list)
#write the data
for k in range(df_size):   #at most matrix_size*matrix_size
    i = tazdictrow[df_trip.iloc[k]['orig_taz']]
    j = tazdictrow[df_trip.iloc[k]['dest_taz']]
  
    output_demand[i][j] = df_trip.iloc[k]['total']
    print('Row: ',df_trip.iloc[k]['orig_taz'],i,"  Col: ",df_trip.iloc[k]['dest_taz'],j," Output",output_demand[i][j],file=outdebugfile)
   
f_output['matrix'] = output_demand   #puts the output_demand array into the omx matrix
f_output.close()
outdebugfile.close()
#You may stop here

In [9]:
# Not needed except for debugging
#Read the input omx trip table
infile = os.path.join(run_folder, '0_tntp_data' ,'demand.omx') 
f_input = omx.open_file(infile)
m1 = f_input['matrix']
input_demand = np.array(m1)

print('Shape:',f_input.shape())
print('Number of tables',len(f_input))
print('Table names:',f_input.list_matrices())
print('attributes:',f_input.list_all_attributes())
print('sum of trips',np.sum(m1))
f_input.close()

Shape: (449, 449)
Number of tables 1
Table names: ['matrix']
attributes: []
sum of trips 32041.0
